## Bases de Datos Relacionales

En esta sección veremos como trabajar con bases de datos relacionales dentro de Python.

Algunas de las ventajas de trabajar con bases de datos en Python son las siguientes:

1. La portabilidad de Python. i.e. funciona en diferentes sistemas: IOS, Windows, Unix, etc.
2. la posibilidad de acceder a _queries_ de las bases de datos por medio de _CONECTORES_
3. Python nos permite conectarnos a las bases de datos más comúnmente usadas:
    * MySQL,
    * PostgreSQL,
    * Oracle,
    * MS-SQL,
    * etc.

## MySQL

Durante esta sección del curso trabajaremos con MySQL. MySQL es uno de los manejadores de bases de datos relacionales más populares.

MySQL, así como otros manejadores de bases de datos, requieren de sistemas que permitan manipular la información. 

## PyMySQL

PyMySQL es un cliente de MySQL escrito en Python basado en [PEP 249](https://www.python.org/dev/peps/pep-0249/): Especificaciones de API para bases de datos en Python v2.0.

Un cliente de bases de datos, `PyMySQL` en este caso, se conecta a la base de datos y hace los queries o manipulaciones a la base de datos seleccionada.

Durante el resto de esta sección, ocuparemos una base de datos ubicada dentro de [db4free.net](https://db4free.net/signup.php). que contiene las siguientes tablas:

```bash
mysql> use pruebas 
Reading table information for completion of table and column names
You can turn off this feature to get a quicker startup with -A

Database changed
mysql> show tables;
+------------------------+
| Tables |
+------------------------+
| customers              |
| employees              |
| offices                |
| orderdetails           |
| orders                 |
| payments               |
| productlines           |
| products               |
+------------------------+
8 rows in set (0.17 sec)
```

#### Diagrama Relacional BD 'Classic Models'

<img src="erclassics.png" alt="Alt text" style="width: 600px;"/>

In [ ]:
pip install pymysql

In [ ]:
import pymysql
import pandas as pd

pwd = ''
conn = pymysql.connect (host = "db4free.net", 
                        user = "",
                        port = 3306,  #PUERTO: una interfaz de comunicación, el 3306 es el predeterminado para MySQL
                        password = pwd,
                        database = "",
                        charset = "utf8",
                        cursorclass=pymysql.cursors.DictCursor)

In [ ]:
conn.close()

## PyMySQL es una interfaz para conectarse a una BBDD MySQL desde Python
1. Conectarse a una BBDD MySQL necesitamos una conexión (un objeto MySQLConnection)
2. Instanciar un objeto cursor de MySQL (MySQLCursor)
    Cursor: objeto que puede ejecutar operaciones en lenguaje SQL. Es un objeto para interactuar con la BBDD.
3. Utilizar los métodos fetchone(), fetchmany() o fetchall() para traernos los datos.
4. Utilizar el método close() para cerrar la conexión a la BBDD.




Dada una conexión, la manera de interactuar con una base de datos desde `pymysql` es mediante la creación de un cursor, el cuál se puede usar junto con una conexión.

In [ ]:
with conn.cursor() as cursor:
    cursor.execute("SELECT VERSION()")
    version = cursor.fetchone()
print(f"versión de MySQL {version}")

In [ ]:
# f-string: mecanismo para formatear strings que se conoce como "String interpolation" > evaluar expresiones de 
# strings que tienen parámetros embebidos (embedded). 

1. Una vez creado cursor, ejecutamos comandos de MySQL por medio del mètodo execute
2. Un cursor recaba la información dentro de la base de dtos y arroja la información a la sesión de Python por medio
de los MÉTODOS `fetchone` , `fetchmany` o `fetchall`.

In [ ]:
with conn.cursor() as cursor:
    cursor.execute("SHOW DATABASES;")
    dbs = cursor.fetchall()
dbs

In [ ]:
with conn.cursor() as cursor:
    cursor.execute("SHOW TABLES;")
    tables = cursor.fetchall()
tables

In [ ]:
type(tables)

# para ejecutar código SQL desde python, usamos la siguiente estructura general:

query = "Aqui va el Query"

```python
with conn.cursor() as cursor: 
    cursor.execute(query)
    resultado=cursor.fetchmany(10)
```

conn.close()

In [ ]:
#Ejemplo: seleccionar todos los clientes

query = """
SELECT * FROM customers
LIMIT 20;
"""

with conn.cursor() as cursor:
    cursor.execute(query)
    customers = cursor.fetchone()

In [ ]:
customers

In [ ]:
query = """
SELECT * FROM customers
LIMIT 20;
"""

with conn.cursor() as cursor:
    cursor.execute(query)
    customers = cursor.fetchall()

In [ ]:
customers

In [ ]:
#define columnas de customers
custcols = ['customerNumber','customerName','contactLastName','contactFirstName',
            'phone', 'addressLine1','addressLine2','city','state','postalCode','country','salesRepEmployeeNumber',
            'creditLimit']

In [ ]:
df_cust = pd.DataFrame(customers, columns= custcols)
df_cust.shape

In [ ]:
df_cust

In [ ]:
q="""
SELECT * FROM payments;
"""
with conn.cursor() as cursor:
    cursor.execute(q)
    payments = cursor.fetchmany(10) #trae los primeros 10 registros

In [ ]:
payments

In [ ]:
df_pay = pd.DataFrame(payments)
df_pay

Al consultar una base de datos, en pocas ocasiones es necesario adquirir todas las columnas. Por lo que podemos explícitamente seleccionar que columnas ocuparemos de la siguiente manera: 

```SQL
SELECT C0, C1, ..., CN FROM tables
```

In [ ]:
#ejemplo:
q="""
SELECT orderDate, shippedDate, customerNumber FROM orders;
"""

with conn.cursor() as cursor:
    cursor.execute(q)
    orders = cursor.fetchmany(10)

In [ ]:
orders

In [ ]:
df_orders = pd.DataFrame(orders)
df_orders

## WHERE
En casos sobre los cuáles nos interesen ciertas filas en específico, recurrimos al comando `WHERE`

```SQL
SELECT C0, C1 FROM table_name WHERE cond
```

En este caso, `cond` es una condición booleana la cual regresa únicamente los elementos que terminen siendo evaluados como verdaderos.

In [ ]:
# seleccionar órdenes que fueron enviadas en un lapso menor a 30 días de realizadas.
q="""
SELECT orderDate, shippedDate, customerNumber
FROM orders
WHERE shippedDate - orderDate < 30;
"""

with conn.cursor() as cursor:
    cursor.execute(q)
    res = cursor.fetchmany(10)

df = pd.DataFrame(res)
df

In [ ]:
conn.close()
pwd = ''
conn = pymysql.connect (host = "db4free.net", 
                        user = "",
                        port = 3306,  #PUERTO: una interfaz de comunicación, el 3306 es el predeterminado para MySQL
                        password = pwd,
                        database = "",
                        charset = "utf8",
                        cursorclass=pymysql.cursors.DictCursor)

## GROUP BY
Podemos realizar consultas sobre un grupo de elementos y aplicar alguna función sobre cada uno de los grupos encontrados por medio del comando `GROUP BY`.

```SQL
SELECT F(C1) FROM table_name GROUP BY C1
```

Cualquier función que podomas ocupar una vez agregados elementos tiene por nombre _aggregate function_. Algunas de las funciones agregadas que podemos usar en MySQL incluyen: 

* `AVG`
* `COUNT`
* `SUM`
* `MAX`
* `MIN`
* `STD`

In [ ]:
df_cust.head()

In [ ]:
# contar clientes por país, seleccionando sólo a Francia, USA, Australia, Alemania y Noruega
q="""
SELECT country, COUNT(customerNumber) AS num_of_customers
FROM customers
WHERE country IN ('France','USA','Australia', 'Germany', 'Norway')
GROUP BY country;
"""
with conn.cursor() as cursor:
    cursor.execute(q)
    res = cursor.fetchall()
df = pd.DataFrame(res)
df


## HAVING

In [ ]:
#HAVING opera sobre el resultado de un agregado (GROUP BY)


In [ ]:
#Sup queremos aquellos países con más de 10 clientes
q="""
SELECT country, COUNT(customerNumber) AS num_of_customers
FROM customers
WHERE country IN ('France','USA','Australia', 'Germany', 'Norway')
GROUP BY country
HAVING num_of_customers > 10; -- HAVING actua sobre el resultado del GROUP BY
"""

with conn.cursor() as cursor:
    cursor.execute(q)
    res = cursor.fetchall()
res

In [ ]:
#conn.close()

In [ ]:
#igual que el anterior, SIN ALIAS DEL COUNT()
q="""
SELECT country, COUNT(customerNumber)
FROM customers
WHERE country IN ('France','USA','Australia', 'Germany', 'Norway')
GROUP BY country
HAVING COUNT(customerNumber) > 10; -- HAVING actua sobre el resultado del GROUP BY
"""
with conn.cursor() as cursor:
    cursor.execute(q)
    res = cursor.fetchall()
res


In [ ]:
#no es necesario la palabra "AS" para definir el alias
q="""
SELECT country, COUNT(customerNumber) num_of_customers
FROM customers
WHERE country IN ('France','USA','Australia', 'Germany', 'Norway')
GROUP BY country
HAVING num_of_customers > 10; -- HAVING actua sobre el resultado del GROUP BY
"""
with conn.cursor() as cursor:
    cursor.execute(q)
    res = cursor.fetchall()
res



### ORDER

In [ ]:
# contar clientes por país, seleccionando sólo a Francia, USA, Australia, Alemania y Noruega. Además quiero ordenar el resultado de 
# forma descendente:

q="""
SELECT country, COUNT(customerNumber) num_of_customers
FROM customers
WHERE country IN ('France', 'USA', 'Australia', 'Germany', 'Norway')
GROUP BY country
HAVING num_of_customers > 3  -- HAVING actua sobre el resultado del GROUP BY
ORDER BY num_of_customers DESC; -- si omitimos "DESC", ordena por default de forma ascendente. 
"""

with conn.cursor() as cursor:
    cursor.execute(q)
    res = cursor.fetchall()

for element in res:
    print(element)

In [ ]:
q="""
SELECT country, COUNT(customerNumber) num_of_customers
FROM customers
WHERE country IN ('France', 'USA', 'Australia', 'Germany', 'Norway')
GROUP BY country
HAVING num_of_customers > 3  -- HAVING actua sobre el resultado del GROUP BY
ORDER BY num_of_customers; -- si omitimos "DESC", ordena por default de forma ascendente. 
"""

with conn.cursor() as cursor:
    cursor.execute(q)
    res = cursor.fetchall()

for element in res:
    print(element)

In [ ]:
# queremos traernos los primeros 20 registros de la tabla customers:
q="""
SELECT * FROM customers
LIMIT 20;
"""

with conn.cursor() as cursor:
    cursor.execute(q)
    res = cursor.fetchall()
res

In [ ]:
df_cust_20 = pd.DataFrame(res)
df_cust_20

In [ ]:
# En GROUP BY y ORDER, podemos referenciar las variables por el orden en el que aparecen 
#en la sentencia SELECT:

q="""
SELECT country, COUNT(customerNumber) num_of_customers
FROM customers
WHERE country IN ('France', 'USA', 'Australia', 'Germany', 'Norway')
GROUP BY 1
HAVING num_of_customers > 3  -- HAVING actua sobre el resultado del GROUP BY
ORDER BY 2 DESC; -- si omitimos "DESC", ordena por default de forma ascendente. 
"""

with conn.cursor() as cursor:
    cursor.execute(q)
    res = cursor.fetchall()

for element in res:
    print(element)

In [ ]:
q="""
SELECT country, COUNT(customerNumber) num_of_customers
FROM customers
WHERE country IN ('France', 'USA', 'Australia', 'Germany', 'Norway')
GROUP BY 1
HAVING num_of_customers > 3  -- HAVING actua sobre el resultado del GROUP BY
ORDER BY 2; -- si omitimos "DESC", ordena por default de forma ascendente. 
"""

with conn.cursor() as cursor:
    cursor.execute(q)
    res = cursor.fetchall()

for element in res:
    print(element)

In [ ]:
q="""
SELECT country, COUNT(customerNumber) num_of_customers
FROM customers
WHERE country IN ('France', 'USA', 'Australia', 'Germany', 'Norway')
GROUP BY 1
HAVING num_of_customers > 3  -- HAVING actua sobre el resultado del GROUP BY
ORDER BY 1 DESC; -- si omitimos "DESC", ordena por default de forma ascendente. 
"""

with conn.cursor() as cursor:
    cursor.execute(q)
    res = cursor.fetchall()

for element in res:
    print(element)

## LIMIT

In [ ]:
#FINALMENTE, usar LIMIT para quedarnos con un número determinado de registros

q="""
SELECT country, COUNT(customerNumber) num_of_customers
FROM customers
WHERE country IN ('France', 'USA', 'Australia', 'Germany', 'Norway')
GROUP BY 1
HAVING num_of_customers > 3  -- HAVING actua sobre el resultado del GROUP BY
ORDER BY 2 DESC -- si omitimos "DESC", ordena por default de forma ascendente.
LIMIT 2;
"""

with conn.cursor() as cursor:
    cursor.execute(q)
    res = cursor.fetchall()

for element in res:
    print(element)

## ¿Cuál es el orden de las operaciones realizadas en el query pasado?

1. `FROM`:     Determina las tablas fuente
2. `WHERE`:    Filtra los registros según condición indicada
3. `GROUP BY`: Agrupa los registros filtrados
4. `HAVING`:   Filtra los grupos resultantes del GROUP BY
5. `SELECT`:   Determina qué columnas y resultados calculados son incluidos en el output.
6. `ORDER BY`: Ordena el conjunto resultante
7. `LIMIT`:    limita el número de filas.

In [ ]:
conn.close()

## Ahora veamos una forma de hacer queries sobre un DF usando SQL en Python

In [1]:
pip install pandasql

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.8 MB/s eta 0:00:0000:0100:01
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26771 sha256=f48999603e731b04eec5268881b9adcfa641de33280a8c57017baf64870df3ba
  Stored in directory: /Users/alexiscaballero/Library/Caches/pip/wheels/68/5d/a5/edc271b998f909801d7956959f699b976cc9896075dc47c153
Successfully built pandasql
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pandasql import sqldf
import pandas as pd

`sqldf` es una función de la biblioteca `pandasql` que permite  ejecutar consultas SQL en DataFrames de pandas.  
* Te permite tratar un DataFrame como si fuera una tabla de base de datos y usar la sintaxis SQL para realizar operaciones de manipulación de datos.
* Proporciona un puente entre el mundo de SQL y pandas, aprovechando el poder de ambos.

In [4]:
departments = pd.read_csv('departments.csv')
employees = pd.read_csv('upd_empl_2.csv')
regions = pd.read_csv('regions.csv')

In [5]:
departments.head()

,department,division
0,Clothing,Home
1,Grocery,Home
2,Decor,Home
3,Furniture,Home
4,Computers,Electronics


In [6]:
employees.head()

,employee_id,first_name,last_name,email,hire_date,department,gender,salary,region_id
0,1,Berrie,Manueau,berr@aol.com.mx,2006-04-20,Sports,F,154864,4
1,2,Aeriell,McNee,aeri@gmail.co,2009-01-26,Tools,F,56752,3
2,3,Sydney,Symonds,sydn@terra.mx,2010-05-17,Clothing,F,95313,4
3,4,Avrom,Rowantree,avro@terra.mx,2014-08-02,Phones & Tablets,M,119674,7
4,5,Fraiser,Morffew,feli@yahoo.com.mx,2003-01-14,Computers,M,55307,5


In [7]:
regions.head()

,region_id,region,country
0,1,Southwest,United States
1,2,Northeast,United States
2,3,Northwest,United States
3,4,Central,Asia
4,5,East Asia,Asia


In [8]:
employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   employee_id  1000 non-null   int64 
 1   first_name   1000 non-null   object
 2   last_name    1000 non-null   object
 3   email        1000 non-null   object
 4   hire_date    1000 non-null   object
 5   department   1000 non-null   object
 6   gender       1000 non-null   object
 7   salary       1000 non-null   int64 
 8   region_id    1000 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 70.4+ KB


In [9]:
q="""
SELECT * FROM regions;
"""

sqldf(q, globals())

,region_id,region,country
0,1,Southwest,United States
1,2,Northeast,United States
2,3,Northwest,United States
3,4,Central,Asia
4,5,East Asia,Asia
5,6,Quebec,Canada
6,7,Nova Scotia,Canada


In [10]:
def fsql(q):
    return sqldf(q, globals())

In [11]:
q="""
SELECT * FROM regions;
"""
fsql(q)

,region_id,region,country
0,1,Southwest,United States
1,2,Northeast,United States
2,3,Northwest,United States
3,4,Central,Asia
4,5,East Asia,Asia
5,6,Quebec,Canada
6,7,Nova Scotia,Canada


In [12]:
employees.head(4)

,employee_id,first_name,last_name,email,hire_date,department,gender,salary,region_id
0,1,Berrie,Manueau,berr@aol.com.mx,2006-04-20,Sports,F,154864,4
1,2,Aeriell,McNee,aeri@gmail.co,2009-01-26,Tools,F,56752,3
2,3,Sydney,Symonds,sydn@terra.mx,2010-05-17,Clothing,F,95313,4
3,4,Avrom,Rowantree,avro@terra.mx,2014-08-02,Phones & Tablets,M,119674,7


In [16]:
# Quiero el promedio del salario de todos los empleados:
q="""
SELECT ROUND(AVG(salary)) 
FROM employees;  -- ROUND es para redondear
"""
fsql(q)

,ROUND(AVG(salary))
0,91572.0


In [20]:
q="""
SELECT ROUND(AVG(salary),1) salario_promedio
FROM employees;
"""
fsql(q)

,salario_promedio
0,91571.6


## Subconsulta i.e.: Subqueries

In [21]:
#Podemos usar una consulta como una fuente de informaciòn:

#Supongamos quiero ek promedio del salario de los empleados PERO quiero excluir a los empleados con el máximo y el mínimo salario:

q="""
SELECT AVG(salary)
FROM employees
WHERE salary NOT IN (                          -- una lista con el max y el min
         (SELECT MAX(salary) FROM employees),  -- aqui el máximo
         (SELECT MIN(salary) FROM employees)   -- aquí el mínimo
);
"""




fsql(q)


,AVG(salary)
0,91567.210421


In [22]:
employees.head(3)

,employee_id,first_name,last_name,email,hire_date,department,gender,salary,region_id
0,1,Berrie,Manueau,berr@aol.com.mx,2006-04-20,Sports,F,154864,4
1,2,Aeriell,McNee,aeri@gmail.co,2009-01-26,Tools,F,56752,3
2,3,Sydney,Symonds,sydn@terra.mx,2010-05-17,Clothing,F,95313,4


In [23]:
departments

,department,division
0,Clothing,Home
1,Grocery,Home
2,Decor,Home
3,Furniture,Home
4,Computers,Electronics
5,Device Repair,Electronics
6,Phones & Tablets,Electronics
7,Garden,Outdoors
8,Camping & Fishing,Outdoors
9,Sports,Outdoors


In [24]:
#sUP quiero a los empleados asignados a un departamento que NO está en la tabla de departamentos:

q="""
SELECT * FROM employees
WHERE department NOT IN (SELECT department FROM departments)
LIMIT 10;
"""

fsql(q)

,employee_id,first_name,last_name,email,hire_date,department,gender,salary,region_id
0,82,Dani,Diter,dani@yahoo.com.mx,2006-01-13,Camping,M,102516,2
1,86,Doll,Blondel,doll@yahoo.com.mx,2007-03-30,Camping,F,136773,7
2,91,Riley,Yellowlea,rile@yahoo.com.mx,2007-03-20,Camping,M,166569,5
3,130,Vincents,Gullivent,vinc@gmail.co,2010-07-24,Camping,M,57696,6
4,140,Eugenia,Scourgie,euge@aol.com.mx,2011-06-29,Camping,F,26747,5
5,181,Jayme,Jimmison,jaym@aol.com,2009-07-18,Camping,M,79893,1
6,186,Rory,Domel,rory@gmail.com,2007-08-30,Camping,F,153133,1
7,203,Lindsy,Coventry,lind@terra.mx,2015-01-18,Maintenance,F,42370,4
8,209,Tabbi,Driuzzi,tabb@gmail.co,2003-12-22,Camping,F,121072,7
9,212,Izabel,Lowthorpe,izab@aol.com,2003-09-07,Camping,F,142059,4


In [27]:
#comprobar lo anterior
q="""
SELECT * FROM departments
WHERE department IN ("Camping", "Maintenance");
"""

fsql(q) #tabla vacía

,department,division


In [ ]:
#Quiero todos los empleados que trabajan en la división de "Electronics":


In [28]:
q="""
SELECT * FROM employees
WHERE department IN (SELECT department FROM departments WHERE division = 'Electronics');
"""

fsql(q)

,employee_id,first_name,last_name,email,hire_date,department,gender,salary,region_id
0,4,Avrom,Rowantree,avro@terra.mx,2014-08-02,Phones & Tablets,M,119674,7
1,5,Fraiser,Morffew,feli@yahoo.com.mx,2003-01-14,Computers,M,55307,5
2,8,Seline,Dubber,seli@hotmail.com,2012-05-28,Phones & Tablets,F,101066,3
3,19,Bernardine,Hendricks,bern@hotmail.com,2006-02-04,Device Repair,F,158268,3
4,20,Jessey,Colum,jess@aol.com.mx,2013-07-08,Computers,M,77173,7
...,...,...,...,...,...,...,...,...,...
128,986,Sheila,Matthiae,shei@yahoo.com,2010-03-26,Computers,F,23878,1
129,989,Ari,Queripel,ari@gmail.co,2013-08-30,Phones & Tablets,M,80695,1
130,992,Katharine,Brussels,kath@excite.com,2003-10-02,Phones & Tablets,F,29806,5
131,994,Aurlie,Kindleysides,aurl@yahoo.com.mx,2004-05-20,Device Repair,F,104822,3


In [29]:
#nulos --> para filtrar los NO NULOS:
q="SELECT * FROM employees WHERE department IS NOT NULL;"

fsql(q)

,employee_id,first_name,last_name,email,hire_date,department,gender,salary,region_id
0,1,Berrie,Manueau,berr@aol.com.mx,2006-04-20,Sports,F,154864,4
1,2,Aeriell,McNee,aeri@gmail.co,2009-01-26,Tools,F,56752,3
2,3,Sydney,Symonds,sydn@terra.mx,2010-05-17,Clothing,F,95313,4
3,4,Avrom,Rowantree,avro@terra.mx,2014-08-02,Phones & Tablets,M,119674,7
4,5,Fraiser,Morffew,feli@yahoo.com.mx,2003-01-14,Computers,M,55307,5
...,...,...,...,...,...,...,...,...,...
995,996,Gardiner,Aron,gard@gmail.com,2015-11-29,Garden,M,111859,1
996,997,Rhianna,Trynor,rhia@aol.com,2005-04-07,Beauty,F,120753,7
997,998,Brandice,Gillicuddy,bran@yahoo.com,2014-11-30,Phones & Tablets,F,134058,2
998,999,Kingston,Piwall,king@hotmail.com,2012-07-07,Music,M,45679,7


In [31]:
regions

,region_id,region,country
0,1,Southwest,United States
1,2,Northeast,United States
2,3,Northwest,United States
3,4,Central,Asia
4,5,East Asia,Asia
5,6,Quebec,Canada
6,7,Nova Scotia,Canada


In [33]:
# quiero a los empleados que trabajan en USA o Canada Y que ganan más de 120k USD:
q="""
SELECT * FROM employees
WHERE salary > 120000
AND region_id IN (SELECT region_id FROM regions WHERE country IN ('United States','Canada'));
"""
fsql(q)

,employee_id,first_name,last_name,email,hire_date,department,gender,salary,region_id
0,6,Bethena,Trow,beth@yahoo.com,2003-06-08,Sports,F,134501,3
1,11,Nickey,Pointon,nick@gmail.co,2006-12-30,Jewelry,M,126333,7
2,14,Jodi,Hook,jodi@aol.com.mx,2003-10-16,Tools,F,126588,2
3,19,Bernardine,Hendricks,bern@hotmail.com,2006-02-04,Device Repair,F,158268,3
4,21,Bernardo,Davage,bern@gmail.co,2013-07-11,Clothing,M,124949,6
...,...,...,...,...,...,...,...,...,...
213,972,Cirstoforo,Mulloch,cirs@aol.com,2011-05-09,Clothing,M,141256,7
214,974,Thorpe,Bick,thor@yahoo.com,2008-01-19,Garden,M,121211,1
215,987,Granny,Barhem,gran@excite.com,2003-05-13,Movies,M,125798,1
216,997,Rhianna,Trynor,rhia@aol.com,2005-04-07,Beauty,F,120753,7


In [34]:
employees.head(3)

,employee_id,first_name,last_name,email,hire_date,department,gender,salary,region_id
0,1,Berrie,Manueau,berr@aol.com.mx,2006-04-20,Sports,F,154864,4
1,2,Aeriell,McNee,aeri@gmail.co,2009-01-26,Tools,F,56752,3
2,3,Sydney,Symonds,sydn@terra.mx,2010-05-17,Clothing,F,95313,4


In [ ]:
## Si utilizo un alias para una variable, necesito referenciar dicho alias en el query externo:

q="""
SELECT a.nombre_empl, a.salario_anual
FROM (SELECT first_name nombre_empl, salary salario_anual
      FROM employees
      WHERE salary > 150000) a;
"""

fsql(q)

## Case

In [36]:
# Sup queremos crear una variable 'estatus_pago' con base en la siguiente definición:

#salary < 90000 --> 'bajo'
# salary >= 90000 AND salary < 150000 --> 'bien pagado'
# en otro caso: 'muy bien pagado'

# --> sentencia CASE:

q="""
SELECT first_name, salary,
CASE
    WHEN salary < 90000 THEN 'BAJO'
    WHEN salary >= 90000 AND salary < 150000 THEN 'BIEN PAGADO'
    ELSE 'muy bien pagado'
END AS estatus_pago                 -- Termina con END AS nombre_de_variable
FROM employees
ORDER BY salary DESC;
"""
df = fsql(q)

In [37]:
mask = df['estatus_pago'] == 'BIEN PAGADO'
df[mask]

,first_name,salary,estatus_pago
121,Christine,149864,BIEN PAGADO
122,Say,149707,BIEN PAGADO
123,Annie,149161,BIEN PAGADO
124,Kaleb,149076,BIEN PAGADO
125,Fiona,149004,BIEN PAGADO
...,...,...,...
491,Carce,91026,BIEN PAGADO
492,Alister,90751,BIEN PAGADO
493,Matt,90428,BIEN PAGADO
494,Maximilianus,90399,BIEN PAGADO


In [38]:
df['estatus_pago'].value_counts()

estatus_pago
BAJO               504
BIEN PAGADO        375
muy bien pagado    121
Name: count, dtype: int64